In [1]:
# imports principais
import polars as pl
import pandas as pd
import numpy as np
import lightgbm as lgb
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Funções Genéricas
- Essas funções formam o “kit básico” de *ingestão* e *EDA* do projeto. Elas deixam o notebook mais limpo, evitam código repetido e permitem ajustes centralizados caso caminhos ou formatos mudem.


In [2]:
DATA_DIR_PARQUET = Path("data/parquet_files/train")
def read_parquet_dir(name: str) -> pl.DataFrame:
    return pl.read_parquet(DATA_DIR_PARQUET / name)

def ver_dataframe(dataframe: pd):
    # Ver o número de linhas e colunas
    print("Tamanho da tabela:", dataframe.shape)

    # Ver os nomes das primeiras 20 colunas e seus tipos
    for nome, tipo in zip(dataframe.columns[:20], dataframe.dtypes[:20]):
        print(f"{nome:40} → {tipo}")
    
    # Conta quantos valores vazios existem em cada coluna
    faltantes = dataframe.null_count()
    print(f"\nValores Nulos por Coluna\n{faltantes}")
    
    print(dataframe.head(5))

def ver_coluna_df(dataframe: pd, coluna, lines=5):
    # Conferir rapidamente
    print(dataframe["date_decision"].head(lines))
    print("Tipo da coluna agora:", dataframe["date_decision"].dtype)
    
# Converter datas de texto/inteiro para Date
def converter_texto_data(dataframe: pd, date_cols: list):
    new_dataframe = dataframe
    for col in date_cols:
        new_dataframe = new_dataframe.with_columns(
            pl.col(col)
            .str.strptime(pl.Date, "%Y-%m-%d", strict=False)  # ajuste formato se precisar
            .alias(col)
        )
    return new_dataframe
    
def ver_dist_column(dataframe: pd, columns):
    dist_target = dataframe.select(pl.col(columns).value_counts())
    print(dist_target)
    

## Importação do DataFrame Meste

In [3]:
def load_master_dataframe(verbose: bool = True) -> pl.DataFrame:
    """
    Carrega o arquivo principal (train_base.parquet),
    converte a coluna date_decision para tipo Date
    e faz uma inspeção básica opcional.

    Parâmetros
    ----------
    verbose : bool (default=True)
        Se True, imprime:
        • shape, tipos e nulos            (ver_dataframe)
        • amostra e dtype de date_decision (ver_coluna_df)
        • distribuição do target           (ver_dist_column)

    Retorno
    -------
    pl.DataFrame
        DataFrame já limpo e pronto para receber features.
    """
    # 1. Lê o arquivo Parquet base
    df = read_parquet_dir("train_base.parquet")

    # 2. Converte a coluna de data
    df = converter_texto_data(df, ["date_decision"])

    # 3. EDA rápida (opcional)
    if verbose:
        ver_dataframe(df)
        ver_coluna_df(df, "date_decision")
        ver_dist_column(df, "target")

    return df


In [4]:
df = load_master_dataframe()

Tamanho da tabela: (1526659, 5)
case_id                                  → Int64
date_decision                            → Date
MONTH                                    → Int64
WEEK_NUM                                 → Int64
target                                   → Int64

Valores Nulos por Coluna
shape: (1, 5)
┌─────────┬───────────────┬───────┬──────────┬────────┐
│ case_id ┆ date_decision ┆ MONTH ┆ WEEK_NUM ┆ target │
│ ---     ┆ ---           ┆ ---   ┆ ---      ┆ ---    │
│ u32     ┆ u32           ┆ u32   ┆ u32      ┆ u32    │
╞═════════╪═══════════════╪═══════╪══════════╪════════╡
│ 0       ┆ 0             ┆ 0     ┆ 0        ┆ 0      │
└─────────┴───────────────┴───────┴──────────┴────────┘
shape: (5, 5)
┌─────────┬───────────────┬────────┬──────────┬────────┐
│ case_id ┆ date_decision ┆ MONTH  ┆ WEEK_NUM ┆ target │
│ ---     ┆ ---           ┆ ---    ┆ ---      ┆ ---    │
│ i64     ┆ date          ┆ i64    ┆ i64      ┆ i64    │
╞═════════╪═══════════════╪════════╪══════════╪═══

## Depósito features

In [5]:
def add_deposit_features(df: pl.DataFrame) -> pl.DataFrame:
    # 1. Carrega os dados de depósito
    dep = read_parquet_dir("train_deposit_1.parquet")
    
    # 2. Converte colunas de data
    dep = converter_texto_data(dep, ["openingdate_313D", "contractenddate_991D"])
    
    # 3. Tratar nulos: flag + preenchimento provisório
    dep = dep.with_columns([
        pl.col("contractenddate_991D").is_null().cast(pl.Int8).alias("dep_active_flag"),
        pl.when(pl.col("contractenddate_991D").is_null())
          .then(pl.col("openingdate_313D"))
          .otherwise(pl.col("contractenddate_991D"))
          .alias("contractenddate_991D")
    ])
    
    # 4. Cria coluna de duração do contrato
    dep = dep.with_columns(
        (pl.col("contractenddate_991D") - pl.col("openingdate_313D"))
        .dt.total_days()
        .alias("dep_contract_duration")
    )
    
    # 5. Agregação por cliente
    dep_agg = dep.group_by("case_id").agg([
        pl.col("amount_416A").mean().alias("dep_amt_mean"),
        pl.col("amount_416A").max().alias("dep_amt_max"),
        pl.count().alias("dep_ops_cnt"),
        pl.col("dep_active_flag").max().alias("dep_has_active"),
        pl.col("dep_contract_duration").mean().alias("dep_dur_mean")
    ])
    
    # 6. Junta com a base principal
    df = df.join(dep_agg, on="case_id", how="left")
    
    return df


In [6]:
df = add_deposit_features(df)
ver_dataframe(df)

Tamanho da tabela: (1526659, 10)
case_id                                  → Int64
date_decision                            → Date
MONTH                                    → Int64
WEEK_NUM                                 → Int64
target                                   → Int64
dep_amt_mean                             → Float64
dep_amt_max                              → Float64
dep_ops_cnt                              → UInt32
dep_has_active                           → Int8
dep_dur_mean                             → Float64

Valores Nulos por Coluna
shape: (1, 10)
┌─────────┬─────────────┬───────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ case_id ┆ date_decisi ┆ MONTH ┆ WEEK_NUM ┆ … ┆ dep_amt_ma ┆ dep_ops_cn ┆ dep_has_ac ┆ dep_dur_me │
│ ---     ┆ on          ┆ ---   ┆ ---      ┆   ┆ x          ┆ t          ┆ tive       ┆ an         │
│ u32     ┆ ---         ┆ u32   ┆ u32      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│         ┆ u32         ┆  

C:\Users\kelve\AppData\Local\Temp\ipykernel_9740\1673889712.py:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("dep_ops_cnt"),


## Person features

In [7]:
def add_person1_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Extrai e agrega informações de train_person_1.parquet:
      - idade média
      - número médio de filhos
      - % de registros femininos
      - quantidade de registros
    """
    person = read_parquet_dir("train_person_1.parquet")

    # 1. Converter coluna de nascimento para Date
    person = converter_texto_data(person, ["birth_259D"])

    # 2. Novas colunas auxiliares
    person = person.with_columns([
        (pl.lit(2025) - pl.col("birth_259D").dt.year()).alias("idade"),
        (pl.col("gender_992L") == "F").cast(pl.Int8).alias("is_female")
    ])

    # 3. Agregação por case_id
    person_agg = (
        person.group_by("case_id")
              .agg([
                  pl.col("idade").mean().alias("person_age_mean"),
                  pl.col("childnum_185L").mean().alias("person_child_avg"),
                  pl.col("is_female").mean().alias("person_pct_female"),
                  pl.count().alias("person_record_count")
              ])
    )

    # 4. Join com a base principal
    df = df.join(person_agg, on="case_id", how="left")

    # 5. Preencher nulos apenas nas colunas criadas
    fill_cols = [
        "person_age_mean",
        "person_child_avg",
        "person_pct_female",
        "person_record_count"
    ]
    df = df.with_columns([
        pl.col(c).fill_null(0).alias(c) for c in fill_cols
    ])

    return df


In [8]:
df = add_person1_features(df)
ver_dataframe(df)

C:\Users\kelve\AppData\Local\Temp\ipykernel_9740\3021984987.py:27: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("person_record_count")


Tamanho da tabela: (1526659, 14)
case_id                                  → Int64
date_decision                            → Date
MONTH                                    → Int64
WEEK_NUM                                 → Int64
target                                   → Int64
dep_amt_mean                             → Float64
dep_amt_max                              → Float64
dep_ops_cnt                              → UInt32
dep_has_active                           → Int8
dep_dur_mean                             → Float64
person_age_mean                          → Float64
person_child_avg                         → Float64
person_pct_female                        → Float64
person_record_count                      → UInt32

Valores Nulos por Coluna
shape: (1, 14)
┌─────────┬─────────────┬───────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ case_id ┆ date_decisi ┆ MONTH ┆ WEEK_NUM ┆ … ┆ person_age ┆ person_chi ┆ person_pct ┆ person_rec │
│ ---     ┆ on          ┆ 

## Crédito features

In [9]:
def add_credit_bureau_b1_features(df: pl.DataFrame) -> pl.DataFrame:
    """
    Agrega estatísticas do arquivo train_credit_bureau_b_1.parquet
    (depth 1 – Bureau B).
    Cria colunas numéricas e preenche nulos com 0.
    """
    cb = read_parquet_dir("train_credit_bureau_b_1.parquet")

    # 1) Converter datas de contrato
    cb = converter_texto_data(cb, ["contractdate_551D"])

    # 2) Dias desde o contrato mais recente
    latest_date = cb["contractdate_551D"].max()
    cb = cb.with_columns(
        (latest_date - pl.col("contractdate_551D"))
        .dt.total_days()
        .alias("recency_days")
    )

    # 3) Agregação segura (só usa colunas se realmente existirem)
    agg_exprs = [
        pl.col("amount_1115A").sum().alias("cb_b1_amt_sum"),
        pl.col("amount_1115A").max().alias("cb_b1_amt_max"),
        pl.col("recency_days").min().alias("cb_b1_recency_days"),
        pl.count().alias("cb_b1_cnt")
    ]

    if "dpd_550P" in cb.columns:
        agg_exprs.append(pl.col("dpd_550P").mean().alias("dpd_550_mean"))
    if "dpd_733P" in cb.columns:
        agg_exprs.append(pl.col("dpd_733P").mean().alias("dpd_733_mean"))
    if "dpdmax_851P" in cb.columns:
        agg_exprs.append(pl.col("dpdmax_851P").mean().alias("dpdmax_851_mean"))

    if {"debtpastduevalue_732A", "debtvalue_227A"}.issubset(cb.columns):
        agg_exprs.append(
            (pl.col("debtpastduevalue_732A").sum() /
             pl.when(pl.col("debtvalue_227A").sum() == 0)
               .then(1)
               .otherwise(pl.col("debtvalue_227A").sum())
            ).alias("cb_b1_debt_due_ratio")
        )

    cb_agg = cb.group_by("case_id").agg(agg_exprs)

    # 4) Combinar médias em DPD geral e máximo
    dpd_cols = [c for c in ["dpd_550_mean", "dpd_733_mean", "dpdmax_851_mean"] if c in cb_agg.columns]

    if dpd_cols:
        cb_agg = cb_agg.with_columns([
            pl.concat_list([pl.col(c) for c in dpd_cols]).list.mean().alias("cb_b1_dpd_mean"),
            pl.concat_list([pl.col(c) for c in dpd_cols]).list.max().alias("cb_b1_dpd_max")
        ])

    # 5) Juntar na base principal
    df = df.join(cb_agg, on="case_id", how="left")

    # 6) Preencher nulos das novas colunas com 0
    new_cols = [c for c in cb_agg.columns if c != "case_id"]
    df = df.with_columns([pl.col(c).fill_null(0).alias(c) for c in new_cols])

    return df


In [10]:
df = add_credit_bureau_b1_features(df)
ver_dataframe(df)          

Tamanho da tabela: (1526659, 24)
case_id                                  → Int64
date_decision                            → Date
MONTH                                    → Int64
WEEK_NUM                                 → Int64
target                                   → Int64
dep_amt_mean                             → Float64
dep_amt_max                              → Float64
dep_ops_cnt                              → UInt32
dep_has_active                           → Int8
dep_dur_mean                             → Float64
person_age_mean                          → Float64
person_child_avg                         → Float64
person_pct_female                        → Float64
person_record_count                      → UInt32
cb_b1_amt_sum                            → Float64
cb_b1_amt_max                            → Float64
cb_b1_recency_days                       → Int64
cb_b1_cnt                                → UInt32
dpd_550_mean                             → Float64
dpd_733_mean     

C:\Users\kelve\AppData\Local\Temp\ipykernel_9740\1190808682.py:25: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("cb_b1_cnt")


## Funções para Treino

In [11]:
def get_numeric_columns(df: pl.DataFrame, drop: list[str] = ["target"]) -> list[str]:
    return [
        col for col, dtype in zip(df.columns, df.dtypes)
        if dtype in (pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16,
                     pl.UInt32, pl.UInt64, pl.Float32, pl.Float64)
        and col not in drop
    ]

In [12]:
def prepare_train_valid(df: pl.DataFrame, num_cols: list[str], stratify: bool = True):
    import pandas as pd
    from sklearn.model_selection import train_test_split

    X = df.select(num_cols).to_pandas()
    y = df["target"].to_pandas()

    if stratify:
        X_train, X_valid, y_train, y_valid = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=42
        )
    else:
        X_train, X_valid, y_train, y_valid = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

    return X_train, X_valid, y_train, y_valid


In [13]:
def train_lightgbm(X_train, y_train, X_valid, y_valid) -> tuple[lgb.Booster, float, float]:
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score

    train_set = lgb.Dataset(X_train, y_train)
    valid_set = lgb.Dataset(X_valid, y_valid, reference=train_set)

    params = dict(
        objective="binary",
        learning_rate=0.05,
        num_leaves=64,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=1,
        metric="auc",
        verbose=-1,
        is_unbalance=True
    )

    model = lgb.train(
        params,
        train_set,
        valid_sets=[valid_set],
        num_boost_round=500,
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    auc = roc_auc_score(y_valid, pred_valid)
    gini = 2 * auc - 1

    print(f"\nAUC  : {auc:.4f}")
    print(f"Gini : {gini:.4f}")

    return model, auc, gini


## Treino

In [14]:
num_cols = get_numeric_columns(df)
X_train, X_valid, y_train, y_valid = prepare_train_valid(df, num_cols)
model, auc, gini = train_lightgbm(X_train, y_train, X_valid, y_valid)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[125]	valid_0's auc: 0.635081

AUC  : 0.6351
Gini : 0.2702
